In [51]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

import qiskit
from qiskit import QuantumCircuit #Aer, IBMQ,
from qiskit.visualization import plot_histogram
from qiskit import transpile
#from qiskit.providers import fake_provider
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeTorino, FakeBrisbane
from qiskit_ibm_runtime.fake_provider import FakeFez, FakeMarrakesh
#fez, marrakesh are both Heron2
from qiskit.providers.basic_provider import BasicSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


In [2]:
import data_extract_funcs

In [5]:
hdwr_8q_tor1 = pd.read_csv("../Hardware_results/8q/8q_torino1.csv")

In [6]:
#hdwr_8q_tor1.replace('',np.nan)
#hdwr_8q_tor1.fillna(0, inplace=True)
hdwr_8q_tor1.head()

,00000000,00000001,00000010,00000011,00000100,00000101,00000110,00000111,00001000,00001001,...,11110110,11110111,11111000,11111001,11111010,11111011,11111100,11111101,11111110,11111111
0,2987,328,201,46,30,9,17,55,36,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,6
1,2946,323,208,52,31,4,14,58,29,9.0,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,3.0,8
2,2986,281,193,44,32,11,21,49,46,6.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2.0,11
3,2977,301,190,51,29,5,13,57,33,6.0,...,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,NaN,10
4,2950,336,215,44,33,8,15,53,18,1.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,16


## Measuring sparsity of data

In [7]:
non_null_vals = hdwr_8q_tor1.count()
print(non_null_vals)

00000000    50
00000001    50
00000010    50
00000011    50
00000100    50
            ..
11111011    28
11111100     4
11111101    12
11111110    38
11111111    50
Length: 256, dtype: int64


In [4]:
def get_percentage_non_zero(df):
    df.replace(0,'')
    percentage_non_zero = sum(df.count())/(df.size)
    return percentage_non_zero

In [ ]:
percentage_non_zero_8q = sum(non_null_vals)/(hdwr_8q_tor1.size)
print(percentage_non_zero_8q)

0.256875


In [11]:
hdwr_4q_tor1 =  pd.read_csv("../Hardware_results/4q/4q_torino1.csv")
percentage_non_zero_4q = sum(hdwr_4q_tor1.count())/(hdwr_4q_tor1.size)
print(percentage_non_zero_4q)

0.8475877192982456


checking the percentage_non_zero for all data

## Meta_data dataframe

In [53]:
df_meta_data = pd.DataFrame(columns = ['nr_qubits','backend','sim','circuit_type','file_path'])
df_meta_data.head()

,nr_qubits,backend,sim,circuit_type,file_path


1. make file names
2. extract nr qubits etc from file names
3. add directory to file names
4. load as df, get percentage non_null

In [117]:

def extract_cols_from_filename(file_name_str, dir_):
    nr_qubits = re.findall(r'\d+',file_name_str[0:3])[0]
    backend_name_start = len(nr_qubits)+2 #might need to 7 change if 'ibm_..." is included in file name"
    circuit_type = file_name_str[-5]
    is_sim = False
    
    if file_name_str.find("fake")!=-1:
        is_sim = True
    if is_sim:
        backend_name_start = backend_name_start +5
    else:
        backend_name_start = backend_name_start +4
        
    backend_name = file_name_str[backend_name_start:-5]

    file_path = dir_+file_name_str
    
    return [nr_qubits,backend_name,is_sim,circuit_type,file_path]

def add_file_to_meta_df(meta_df,file_name,dir_):
    row = extract_cols_from_filename(file_name,dir_)
    meta_df.loc[len(meta_df)] = row
    
def add_qfolder_to_df(df,folder_dir,backends_,nr_qubits):
    for backend in backends_:
        backend_name = backend.backend_name
        file_names = data_extract_funcs.make_file_names(backend_name,nr_qubits)
        for file_name in file_names:
            add_file_to_meta_df(df,file_name,folder_dir)

def add_qfolders(df,backends,folders):
    for folder in folders:
        nr_qubits = re.findall(r'\d+',folder)[0]
        add_qfolder_to_df(df_meta_data,folder,backends,nr_qubits)

def blank_meta_df():
    meta_df = pd.DataFrame(columns = ['nr_qubits','backend','sim','circuit_type','file_path'])
    return meta_df

def load_meta_df(meta_df,experiment_type):#Hardware,Simulation,Refreshed_Simulation
    dir_Hardware = ["../Hardware_results/4q/","../Hardware_results/8q/"]
    dir_Sims = ["../Simulated_results/4q/","../Simulated_results/8q/","../Simulated_results/16q/"]
    dir_Refr_Sims = ["../Refreshed_Simulated_results/4q/","../Refreshed_Simulated_results/8q/","../Refreshed_Simulated_results/16q/"]
    
    service = QiskitRuntimeService()
    fake_backends = [FakeTorino(), FakeFez(), FakeMarrakesh(),FakeBrisbane()]
    hardware_backends = [service.backend('ibm_brisbane'), service.backend('ibm_torino')]
    
    match experiment_type:
        case 'Hardware':
            dir_ = dir_Hardware
            backends_ = hardware_backends
        case 'Simulation':
            dir_ = dir_Sims
            backends_ = fake_backends
        case 'Refreshed_Simulation':
            dir_ = dir_Refr_Sims
            backends_ = fake_backends

    add_qfolders(meta_df,backends_,dir_)


def get_results_df_from_row(row_index,meta_df):
    csv_file = meta_df.loc[row_index,'file_path']
    csv_df = pd.read_csv(csv_file)
    return csv_df

In [110]:
df_meta_data = blank_meta_df()
df_meta_data.head()

,nr_qubits,backend,sim,circuit_type,file_path


In [116]:
fake_backends = [FakeTorino(), FakeFez(), FakeMarrakesh(),FakeBrisbane()]
for backend in fake_backends:
    backend_name = backend.backend_name
    #print(backend_name)
    file_names = data_extract_funcs.make_file_names(backend_name,4)
    print(file_names)
for file_name in file_names:
    print(file_name)
#add_file_to_meta_df(df,file_name,folder_dir)

['4q_fake_torino1.csv', '4q_fake_torino2.csv', '4q_fake_torino3.csv']
['4q_fake_fez1.csv', '4q_fake_fez2.csv', '4q_fake_fez3.csv']
['4q_fake_marrakesh1.csv', '4q_fake_marrakesh2.csv', '4q_fake_marrakesh3.csv']
['4q_fake_brisbane1.csv', '4q_fake_brisbane2.csv', '4q_fake_brisbane3.csv']
4q_fake_brisbane1.csv
4q_fake_brisbane2.csv
4q_fake_brisbane3.csv


In [118]:
df_meta_data = blank_meta_df()

add_qfolder_to_df(df_meta_data,"../Simulated_results/4q/",fake_backends,4)
df_meta_data.head(20)

,nr_qubits,backend,sim,circuit_type,file_path
0,4,torino,True,1,../Simulated_results/4q/4q_fake_torino1.csv
1,4,torino,True,2,../Simulated_results/4q/4q_fake_torino2.csv
2,4,torino,True,3,../Simulated_results/4q/4q_fake_torino3.csv
3,4,fez,True,1,../Simulated_results/4q/4q_fake_fez1.csv
4,4,fez,True,2,../Simulated_results/4q/4q_fake_fez2.csv
5,4,fez,True,3,../Simulated_results/4q/4q_fake_fez3.csv
6,4,marrakesh,True,1,../Simulated_results/4q/4q_fake_marrakesh1.csv
7,4,marrakesh,True,2,../Simulated_results/4q/4q_fake_marrakesh2.csv
8,4,marrakesh,True,3,../Simulated_results/4q/4q_fake_marrakesh3.csv
9,4,brisbane,True,1,../Simulated_results/4q/4q_fake_brisbane1.csv


In [121]:
df_meta_data = blank_meta_df()
load_meta_df(df_meta_data,'Refreshed_Simulation')
df_meta_data.head(20)

,nr_qubits,backend,sim,circuit_type,file_path
0,4,torino,True,1,../Refreshed_Simulated_results/4q/4q_fake_tori...
1,4,torino,True,2,../Refreshed_Simulated_results/4q/4q_fake_tori...
2,4,torino,True,3,../Refreshed_Simulated_results/4q/4q_fake_tori...
3,4,fez,True,1,../Refreshed_Simulated_results/4q/4q_fake_fez1...
4,4,fez,True,2,../Refreshed_Simulated_results/4q/4q_fake_fez2...
5,4,fez,True,3,../Refreshed_Simulated_results/4q/4q_fake_fez3...
6,4,marrakesh,True,1,../Refreshed_Simulated_results/4q/4q_fake_marr...
7,4,marrakesh,True,2,../Refreshed_Simulated_results/4q/4q_fake_marr...
8,4,marrakesh,True,3,../Refreshed_Simulated_results/4q/4q_fake_marr...
9,4,brisbane,True,1,../Refreshed_Simulated_results/4q/4q_fake_bris...


### Loading the csv results from the file path in the metadata df

In [96]:
row = 2
csv_file1 = df_meta_data.loc[row,'file_path']
print(csv_file1)

../Hardware_results/4q/4q_ibm_torino3.csv


In [ ]:
csv1_df = pd.read_csv(csv_file1)
csv1_df.head()

,0000,0001,0010,0011,0100,0101,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111
0,2963,488,229,77,110,53,16,11,74,50,8,5,4,7,1.0,NaN
1,2923,456,248,99,110,63,18,11,70,61,11,5,8,10,1.0,2.0
2,2963,430,245,78,133,59,18,19,68,42,9,8,12,8,4.0,NaN
3,2909,472,238,98,114,69,19,16,74,44,6,8,8,12,5.0,4.0
4,2919,458,228,98,121,74,27,16,66,50,5,5,15,7,3.0,4.0


## Testing different methods for dealing with sparsity
### Removing null columns

### PCA

In [ ]:
#https://note.nkmk.me/en/python-numpy-zeros-ones-full/